# Défis open data : JOP2024 et offre culturelle

In [1]:
import json

import pandas as pd

## Jeu de données : sites de compétition

In [2]:
with open("datasets/raw/paris-2024-sites-de-competition.json", "r", encoding="utf-8") as f:
    data = json.load(f)

df = pd.json_normalize(data)

In [3]:
df.head(3)

,code_site,nom_site,category_id,sports,start_date,end_date,adress,latitude,longitude,point_geo.lon,point_geo.lat
0,ALX,Pont Alexandre III,venue-olympic,Cyclisme sur route - arrivée Contre-la-montre ...,2024-07-27,2024-08-09,None,"48,8637","2,3134",2.313400,48.86370
1,BCY,Arena Bercy,venue-olympic,"Basketball (BKB), Gymnastique artistique (GAR)...",2024-07-27,2024-08-11,None,"48,83863","2,378597",2.378597,48.83863
2,CDM,Arena Champ de Mars,venue-olympic,"Judo (JUD), Lutte (WRE)",2024-07-27,2024-08-11,None,"48,8531","2,30252",2.302520,48.85310


In [4]:
df.sample(3)

,code_site,nom_site,category_id,sports,start_date,end_date,adress,latitude,longitude,point_geo.lon,point_geo.lat
14,NPA,Arena Paris Nord,venue-paralympic,Volleyball assis (PVBS),2024-08-29,2024-09-07,None,"48,9721","2,5149",2.514900,48.972100
45,BOR,Stade de Bordeaux,venue-olympic,Football (FBL),2024-07-24,2024-08-02,None,44.8974,-0.5616,-0.561600,44.897400
33,LC2,La Concorde 2,venue-olympic,BMX freestyle (BMF),2024-07-30,2024-07-31,None,"48,86504456","2,32119516",2.321195,48.865045


In [5]:
df.shape

(63, 11)

In [6]:
df.columns

Index(['code_site', 'nom_site', 'category_id', 'sports', 'start_date',
       'end_date', 'adress', 'latitude', 'longitude', 'point_geo.lon',
       'point_geo.lat'],
      dtype='object')

In [7]:
df.dtypes

code_site         object
nom_site          object
category_id       object
sports            object
start_date        object
end_date          object
adress            object
latitude          object
longitude         object
point_geo.lon    float64
point_geo.lat    float64
dtype: object

In [8]:
df.nunique()

code_site        44
nom_site         44
category_id       2
sports           56
start_date       15
end_date         18
adress            0
latitude         43
longitude        43
point_geo.lon    43
point_geo.lat    43
dtype: int64

In [9]:
df.isna().sum()

code_site         0
nom_site          0
category_id       0
sports            0
start_date        0
end_date          0
adress           63
latitude          0
longitude         0
point_geo.lon     0
point_geo.lat     0
dtype: int64

La colonne `code_site`ne sera pas utile, on la supprime.

In [10]:
df = df.drop(columns="code_site")

df.sample(3)

,nom_site,category_id,sports,start_date,end_date,adress,latitude,longitude,point_geo.lon,point_geo.lat
44,Arena Bercy,venue-paralympic,Basketball fauteuil (PWBK),2024-08-29,2024-09-08,None,"48,83863","2,378597",2.378597,48.838630
16,Stade Roland Garros,venue-paralympic,Tennis fauteuil (PWTE),2024-08-30,2024-09-07,None,"48,845968","2,253522",2.253522,48.845968
62,Stade Yves-du-Manoir,venue-olympic,Hockey (HOC),2024-07-27,2024-08-09,None,"48,92934371","2,24777122",2.247771,48.929344


`category_id` n'a que deux valeurs, on supprime le prefix "venue-" et on renomme cette colonne en `games_type`.

In [11]:
df["category_id"] = df["category_id"].str.removeprefix("venue-")
df = df.rename(columns={"category_id": "games_type"})

df.sample(3)

,nom_site,games_type,sports,start_date,end_date,adress,latitude,longitude,point_geo.lon,point_geo.lat
6,Centre national de tir de Châteauroux,paralympic,Para Tir Sportif (PSHO),2024-08-30,2024-09-05,None,"46,8157","1,7582",1.75820,46.8157
29,Paris La Défense Arena,olympic,"Natation (SWM), Water-polo (WPO)",2024-07-27,2024-08-11,None,"48,8954","2,2294",2.22940,48.8954
27,Arena Champ de Mars,paralympic,"Para Judo (PJUD), Rugby fauteuil (PWRU)",2024-08-29,2024-09-07,None,"48,8531","2,30252",2.30252,48.8531


`sports`, semble contenir plusieurs sports pour une même ligne.

In [12]:
df["sports"].unique()[:20]

array(['Cyclisme sur route - arrivée Contre-la-montre (CRD), Natation marathon (OWS), Triathlon (TRI)',
       'Basketball (BKB), Gymnastique artistique (GAR), Gymnastique trampoline (GTR)',
       'Judo (JUD), Lutte (WRE)',
       'Badminton (BDM), Gymnastique rythmique (GRY)',
       'Para badminton (PBDM), Para Haltérophilie (PPWL)',
       'Para cyclisme sur route (PCRD)', 'Para Tir Sportif (PSHO)',
       'Para Natation (PSWM)', 'Cécifoot (PFBB)',
       'Escrime (FEN), Taekwondo (TKW)',
       'Athlétisme - départ Marathon (ATM)', 'Voile (SAL)',
       'Football (FBL)', 'Boxe (BOX), Pentathlon moderne (MDN)',
       'Volleyball assis (PVBS)', 'Tennis fauteuil (PWTE)',
       'Tennis (TEN), Boxe (BOX)', 'Volley-ball (VVO)', 'Boccia (PBOC)',
       'Surf (SRF)'], dtype=object)

Il y a 1, 2, 3 sports ou plus pour une même ligne mais on laisse comme cela. Par contre, on supprime le code du sport qui se trouve entre parenthèses.

In [13]:
df["sports"] = df["sports"].replace(r"\s*\([^)]*\)", "", regex=True)

df.sample(3)

,nom_site,games_type,sports,start_date,end_date,adress,latitude,longitude,point_geo.lon,point_geo.lat
0,Pont Alexandre III,olympic,"Cyclisme sur route - arrivée Contre-la-montre,...",2024-07-27,2024-08-09,None,"48,8637","2,3134",2.313400,48.863700
26,Centre Aquatique,olympic,"Natation artistique, Plongeon, Water-polo",2024-07-27,2024-08-10,None,"48,923723","2,35578",2.355780,48.923723
4,Arena Porte de la Chapelle,paralympic,"Para badminton, Para Haltérophilie",2024-08-29,2024-09-08,None,"48,8997292","2,3605141",2.360514,48.899729


Les dates ne sont pas au bon format, essayons de les convertir pour voir si aucune ne pose problème.

In [14]:
df["start_date"] = pd.to_datetime(df["start_date"])
df["end_date"] = pd.to_datetime(df["end_date"])

df[["start_date", "end_date"]].dtypes

start_date    datetime64[ns]
end_date      datetime64[ns]
dtype: object

La conversion a fonctionné sans erreur.

`adress`est toujours vide, on supprime la colonne.

In [15]:
df = df.drop(columns="adress")

`latitude` et `longitude` ne sont pas gardées :
* en doublon avec `point_geo.lon` et `point_geo.lat` ;
* pas au format numérique.

In [16]:
df = df.drop(columns=["latitude", "longitude"])

df = df.rename(columns={"point_geo.lon": "lon", "point_geo.lat": "lat"})

df.sample(3)

,nom_site,games_type,sports,start_date,end_date,lon,lat
58,Arena Paris Sud 6,olympic,"Handball, Haltérophilie",2024-07-25,2024-08-11,2.290865,48.829381
0,Pont Alexandre III,olympic,"Cyclisme sur route - arrivée Contre-la-montre,...",2024-07-27,2024-08-09,2.313400,48.863700
21,Vélodrome National de Saint-Quentin-en-Yvelines,paralympic,Para Cyclisme sur piste,2024-08-29,2024-09-01,2.034983,48.788010


Le dataset est prêt à être exporté.

In [20]:
df.to_json("datasets/jop2024-competition-sites.json", orient="index", indent=4)